In [3]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import math
import torch.nn.functional as F
import numpy as np

In [21]:
transform = transforms.Compose([
    # rgb to grayscale transform 
    # transform. normalization
    # transform. resize / rescale
    transforms.ToTensor()

])

train_dataset_path = '~/Desktop/research_data/SARscope/train_data'
test_dataset_path = '~/Desktop/research_data/SARscope/test_data'

dataset_train = ImageFolder(root = train_dataset_path, transform = transform)
dataset_test = ImageFolder(root = test_dataset_path, transform = transform)

batch_size = 32
dataloader_train = DataLoader(dataset_train, batch_size = batch_size, shuffle = False)
dataloader_test = DataLoader(dataset_test, batch_size = batch_size, shuffle = True)

for i, data in enumerate(dataloader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    print(labels.sum() == 0)

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)

In [5]:
print("shape: ", dataset_train[0][0].shape)
len(dataloader_train)

shape:  torch.Size([3, 640, 640])


148

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [7]:
print(dataset_train[0][0][1].sum() == dataset_train[0][0][0].sum())

tensor(True)


In [3]:
import torch.nn as nn

In [4]:
def relu(x):
    _relu = nn.ReLU()
    
    return _relu(x)
# change to global min / max
def curly_N(w):
    w_min, w_max = torch.min(torch.min(torch.min(w))), torch.max(torch.max(torch.max(w)))
    reg_N = (w - w_min) / (w_max - w_min)
    return v_p

def curly_Nprime(w):
    w_min, w_max = torch.min(torch.min(torch.min(w))), torch.max(torch.max(torch.max(w)))
    curly_N = (w - w_min + 1) / (w_max - w_min + 2)
    return curly_N
    # return (w - np.min(w) + 1) / (np.max(w) - np.min(w) + 2)

def f_VHN(x, w):
    relu_x = relu(curly_N(x))
    relu_w = relu(curly_Nprime(w))
    
    return relu_x * relu_w

In [5]:
# class MyLinearLayer(nn.Module):
#     """ Custom Linear layer but mimics a standard linear layer """
#     def __init__(self, size_in, size_out):
#         super().__init__()
#         self.size_in, self.size_out = size_in, size_out
#         weights = torch.Tensor(size_out, size_in)
#         self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
#         bias = torch.Tensor(size_out)
#         self.bias = nn.Parameter(bias)

#         # initialize weights and biases
#         nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
#         fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)
#         bound = 1 / math.sqrt(fan_in)
#         nn.init.uniform_(self.bias, -bound, bound)  # bias init

#     def forward(self, x):
#         w_times_x= torch.mm(x, self.weights.t())
#         return torch.add(w_times_x, self.bias)  # w times x + b
        
class VHNLayer(nn.Module):
    """ Custom VHN layer """
    def __init__(self, channels, img_len, img_width):
        super().__init__()
        self.channels, self.img_len, self.img_width = channels, img_len, img_width
        weights = torch.Tensor(channels, img_len, img_width)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        
        

    def forward(self, x):
        
        return f_VHN(x, self.weights) 

In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.vhn1 = VHNLayer(3, 640, 640)
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, 5)
        self.pool = nn.MaxPool2d(3, 3)
        self.fc1 = nn.Linear(28224, 8000)
        self.fc2 = nn.Linear(8000, 84)
        self.fc3 = nn.Linear(84, 16)
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.vhn1(x)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.sigmoid(self.fc4(x))
        # print(x.shape)
        return x


net = ConvNet()


NameError: name 'torch' is not defined

In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()
criterion2 = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


In [7]:
for epoch in range(2):  # loop over the dataset multiple times
    print(epoch)
    running_loss = 0.0
    for i, data in enumerate(dataloader_train, 0):
        # get the inputs; data is a list of [inputs, labels]

        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss1 = criterion(outputs, labels.reshape(32,1).type(torch.float32))
        loss2 = criterion2(net.vhn1.weights, labels.reshape(32,1).type(torch.float32))

        loss = loss1 + loss2
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        print(i)
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

0


NameError: name 'dataloader_train' is not defined

In [16]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in dataloader_test:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        print((predicted == labels).sum())
        correct += (predicted == labels).sum().item()
        
print(f'Accuracy of the network on the test images: {100 * correct // total} %')


tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
Accuracy of the network on the test images: 100 %
